In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, GRU
from keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.utils as ku
tf.config.run_functions_eagerly(True)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-09-01 11:33:25.239586: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-01 11:33:25.250004: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 11:33:25.262625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 11:33:25.266237: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 11:33:25.276681: I tensorflow/core/platform/cpu_feature_guar

Num GPUs Available:  1


I0000 00:00:1725170606.744411   39793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725170606.782592   39793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1725170606.782665   39793 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
rawdataset = pd.read_csv("./dataset/full_dataset.csv")

In [3]:
dataset = rawdataset[:5000]
dataset = dataset.drop(["link", "source"], axis=1)

In [4]:
print("Data shape:", dataset.shape)
dataset.head()

Data shape: (5000, 5)


,Unnamed: 0,title,ingredients,directions,NER
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...","[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....","[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...","[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...","[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...","[""peanut butter"", ""graham cracker crumbs"", ""bu..."


In [5]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation.replace(",", "")).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt

NER = [clean_text(x) for x in dataset.NER]
ingredients = [clean_text(x) for x in dataset.ingredients]
directions = [clean_text(x) for x in dataset.directions]

In [6]:
finaldataset = []
for i in range(len(NER)):
    finaldataset.append("Ingredients needed: "+NER[i]+"\nRecipe: "+directions[i])
#finaldataset.append("~")
max_length = 0
for i in finaldataset:
    max_length = max(max_length, len(i))
print("max_length:", max_length)
print()
print("Recipe at index 0:\n", finaldataset[0])

max_length: 1146

Recipe at index 0:
 Ingredients needed: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits
Recipe: in a heavy 2quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine, stir over medium heat until mixture bubbles all over top, boil and stir 5 minutes more take off heat, stir in vanilla and cereal mix well, using 2 teaspoons, drop and shape into 30 clusters on wax paper, let stand until firm, about 30 minutes


In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    char_level=False,
    filters='',
    lower=False,
    split=' '
)
tokenizer.fit_on_texts(finaldataset)
tokenizer.fit_on_texts(["~"])
vocab_size = len(tokenizer.word_index) + 1
text = ["pineapple condensed milk lemons pecans graham cracker crusts"]
token_list = tokenizer.texts_to_sequences(text)
print(text)
print(token_list)

['pineapple condensed milk lemons pecans graham cracker crusts']
[[221, 333, 99, 3080, 456, 244, 236, 1433]]


In [8]:
finaldataset_token = tokenizer.texts_to_sequences(finaldataset)
print('Vectorized dataset size', len(finaldataset_token))

Vectorized dataset size 5000


In [9]:
tokenizer.sequences_to_texts(finaldataset_token[:2])

['Ingredients needed: brown sugar, milk, vanilla, nuts, butter, bite size shredded rice biscuits\nRecipe: in a heavy 2quart saucepan, mix brown sugar, nuts, evaporated milk and butter or margarine, stir over medium heat until mixture bubbles all over top, boil and stir 5 minutes more take off heat, stir in vanilla and cereal mix well, using 2 teaspoons, drop and shape into 30 clusters on wax paper, let stand until firm, about 30 minutes',
 'Ingredients needed: beef, chicken breasts, cream of mushroom soup, sour cream\nRecipe: place chipped beef on bottom of baking dish, place chicken on top of beef, mix soup and cream together pour over chicken bake, uncovered, at 275u00b0 for 3 hours']

In [10]:
for recipe_index, recipe in enumerate(finaldataset_token[:10]):
    print('Recipe #{} length: {}'.format(recipe_index + 1, len(recipe)))

Recipe #1 length: 73
Recipe #2 length: 40
Recipe #3 length: 41
Recipe #4 length: 74
Recipe #5 length: 47
Recipe #6 length: 166
Recipe #7 length: 53
Recipe #8 length: 46
Recipe #9 length: 58
Recipe #10 length: 75


In [11]:
dataset_vectorized_padded_without_stops = tf.keras.preprocessing.sequence.pad_sequences(
    finaldataset_token,
    padding='post',
    truncating='post',
    # We use -1 here and +1 in the next step to make sure that all recipes will have at least 1 stops
    # sign at the end, since each sequence will be shifted and truncated afterwards (to generate X and Y sequences).
    maxlen=max_length-1,
    value=tokenizer.texts_to_sequences(["~"])[0]
)

In [12]:
dataset_vectorized_padded = tf.keras.preprocessing.sequence.pad_sequences(
    dataset_vectorized_padded_without_stops,
    padding='post',
    truncating='post',
    maxlen=max_length+1,
    value=tokenizer.texts_to_sequences(["~"])[0]
)

In [13]:
for recipe_index, recipe in enumerate(dataset_vectorized_padded[:10]):
    print('Recipe #{} length: {}'.format(recipe_index, len(recipe)))

Recipe #0 length: 1147
Recipe #1 length: 1147
Recipe #2 length: 1147
Recipe #3 length: 1147
Recipe #4 length: 1147
Recipe #5 length: 1147
Recipe #6 length: 1147
Recipe #7 length: 1147
Recipe #8 length: 1147
Recipe #9 length: 1147


In [22]:
tokenizer.sequences_to_texts([dataset_vectorized_padded[1]])

['Ingredients needed: beef, chicken breasts, cream of mushroom soup, sour cream\nRecipe: place chipped beef on bottom of baking dish, place chicken on top of beef, mix soup and cream together pour over chicken bake, uncovered, at 275u00b0 for 3 hours ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~

In [27]:
dataset = tf.data.Dataset.from_tensor_slices(dataset_vectorized_padded)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(1147,), dtype=tf.int32, name=None)>

In [29]:
def split_input_target(recipe):
    input_text = recipe[:-1]
    target_text = recipe[1:]
    
    return input_text, target_text

In [30]:
dataset_targeted = dataset.map(split_input_target)
dataset_targeted

<_MapDataset element_spec=(TensorSpec(shape=(1146,), dtype=tf.int32, name=None), TensorSpec(shape=(1146,), dtype=tf.int32, name=None))>

In [32]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 1000

In [33]:
dataset_train = dataset_targeted \
      .shuffle(SHUFFLE_BUFFER_SIZE) \
      .batch(BATCH_SIZE, drop_remainder=True) \
      .repeat()

In [44]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 100, input_length=max_length))
model.add(GRU(128))
model.add(Dense(1146, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [45]:
print(vocab_size)
model.fit(dataset_train, epochs=10, steps_per_epoch=200, batch_size=32)

6287
Epoch 1/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 24s 118ms/step - loss: 48967452.0000
Epoch 2/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 23s 116ms/step - loss: 49053628.0000
Epoch 3/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 22s 112ms/step - loss: 49023560.0000
Epoch 4/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 23s 114ms/step - loss: 49035628.0000
Epoch 5/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 22s 112ms/step - loss: 49035060.0000
Epoch 6/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 22s 112ms/step - loss: 49046788.0000
Epoch 7/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 23s 116ms/step - loss: 49064824.0000
Epoch 8/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 23s 115ms/step - loss: 49072336.0000
Epoch 9/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 23s 114ms/step - loss: 49129936.0000
Epoch 10/10
200/200 ━━━━━━━━━━━━━━━━━━━━ 23s 113ms/step - loss: 49129548.0000


In [61]:
pred = model.predict(np.array(tokenizer.texts_to_sequences(["brown sugar"])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


TypeError: 'int' object is not iterable